# Bag-of-words model

[위키피디아](https://en.wikipedia.org/wiki/Bag-of-words_model)

다음의 두 문장이 있다고 하자,

(1) John likes to watch movies. Mary likes movies too.   
(2) John also likes to watch football games.   

위 두 문장을 토큰화 하여 가방에 담아주면 다음과 같다.

[
    "John",
    "likes",
    "to",
    "watch",
    "movies",
    "Mary",
    "too",
    "also",
    "football",
    "games"
]


그리고 배열의 순서대로 가방에서 각 토큰이 몇 번 등장하는지 횟수를 세어준다.

(1) [1, 2, 1, 1, 2, 1, 1, 0, 0, 0]   
(2) [1, 1, 1, 1, 0, 0, 0, 1, 1, 1]   

=> **머신러닝 알고리즘이 이해할 수 있는 형태**로 바꿔주는 작업이다.

***
단어 가방을 n-gram을 사용해 bigram 으로 담아주면 다음과 같다.

[
    "John likes",
    "likes to",
    "to watch",
    "watch movies",
    "Mary likes",
    "likes movies",
    "movies too",
]

=> 여기에서는 CountVectorizer를 통해 위 작업을 한다.

# Scikit-learn 의 CountVectorizer 를 통해 Featuer 생성

- 정규표현식을 사용해 토큰을 추출한다.
- 모두 소문자로 변환시키기 때문에 good, Good, gOod이 모두 같은 특성이 된다.
- 의미없는 특성을 많이 생성하기 때문에 적어도 두 개의 문서에 나타난 토큰만을 사용한다. 
    - *min_df* 파라미터로 토큰이 나타날 최소 문서 개수를 지정할 수 있다.

***
**CountVectorizer**
: 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BOW 인코딩한 벡터를 만든다.
- 문서를 토큰 리스트로 변환한다.
- 각 문서에서 토큰의 출현 빈도를 센다.
- 각 문서를 BOW 인코딩 벡터로 변환한다.

*parameter*
- stop_words : 문자열 {‘english’}, 리스트 또는 None (디폴트)
    - stop words 목록.‘english’이면 영어용 스탑 워드 사용.
    - 리스트 형태로 불용어로 처리하고자 하는 문자를 넣어준다
<br><br>   
- analyzer : 문자열 {‘word’, ‘char’, ‘char_wb’} 또는 함수
    - 단어 n-그램, 문자 n-그램, 단어 내의 문자 n-그램
    - 문자열 또는 함수로 어떤 단위로 토큰화 할지 정의
<br><br>        
- token_pattern : string
    - 토큰 정의용 정규 표현식
<br><br>       
- tokenizer : 함수 또는 None (디폴트)
    - 토큰 생성 함수 .
<br><br>        
- ngram_range : (min_n, max_n) 튜플
    - n-그램 범위
<br><br>        
- max_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
    - 단어장에 포함되기 위한 최대 빈도
<br><br>        
- min_df : 정수 또는 [0.0, 1.0] 사이의 실수. 디폴트 1
    - 단어장에 포함되기 위한 최소 빈도
<br><br>        
- max_features : build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

[Scikit-Learn 의 문서 전처리 기능](https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/)

In [1]:
#part1-1 에서 전처리한 텍스트 데이터
import pandas as pd
import pickle

with open('data/p1_clean_train_reviews.p', 'rb') as file:
    clean_train_reviews = pickle.load(file)
    
with open('data/p1_clean_test_reviews.p', 'rb') as file:
    clean_test_reviews = pickle.load(file)

clean_train_reviews

0        stuff go moment mj start listen music watch od...
1        classic war world timothi hine entertain film ...
2        film start manag nichola bell give welcom inve...
3        must assum prais film greatest film opera ever...
4        superbl trashi wondrous unpretenti exploit hoo...
                               ...                        
24995    seem like consider gone imdb review film went ...
24996    believ made film complet unnecessari first fil...
24997    guy loser get girl need build pick stronger su...
24998    minut documentari bu uel made earli one spain ...
24999    saw movi child broke heart stori unfinish end ...
Name: review, Length: 25000, dtype: object

In [2]:
# !pip install sklearn

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

# 튜토리얼과 다르게 파라메터 값을 수정
# 파라메터 값만 수정해도 캐글 스코어 차이가 많이 남
vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = None,
                             preprocessor = None, 
                             stop_words = None, 
                             min_df = 2, # 토큰이 나타날 최소 문서 개수
                             ngram_range=(1, 3), 
                             max_features = 20000 
                            )
vectorizer

CountVectorizer(max_features=20000, min_df=2, ngram_range=(1, 3))

cf. 속도 개선을 위해 pipeline 사용   
참고 : https://stackoverflow.com/questions/28160335/plot-a-document-tfidf-2d-graph


- pipeline 에서 메모리의 캐시를 사용할 수 있다는 문구도 있습니다. 하지만 성능상의 이슈라기보다는 cross-validation 과 GridSearch 과정을 하나로 만들어 주는게 pipeline의 가장 큰 장점이라고 합니다. 

- Countvectorizer 사용시 하나의 과정만 묶어줘서 사실 굳이 pipeline을 쓸 필요는 없는데 보통 transform 하는 과정에서 pipeline을 사용합니다.

- 아래의 코드에선 cross-validation 과 GridSearch 과정이 없는데 해당 과정은 모델의 성능(정확도)를 측정해 보는 과정이고, GridSearch는 최적의 하이퍼파라메터를 찾는 과정입니다.

- Sequentially apply a list of transforms and a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit. The transformers in the pipeline can be cached using memory argument.

- 더 읽어보기 : https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

In [4]:
pipeline = Pipeline([
    ('vect', vectorizer),
])  

Train 데이터 벡터화

보통 fit을 학습하는데 사용하는데 여기에서는 벡터화 할 때도 fit을 사용했습니다. 사이킷런에 구현된 벡터화 알고리즘이 fit을 사용해서 벡터화 하도록 되어 있습니다. 따라서 RF에서의 fit과 벡터화에 사용되는 fit은 다른 성격입니다.

In [5]:
%time train_data_features = pipeline.fit_transform(clean_train_reviews)
train_data_features


CPU times: user 13.3 s, sys: 659 ms, total: 13.9 s
Wall time: 14 s


<25000x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 2762315 stored elements in Compressed Sparse Row format>

In [6]:
train_data_features.shape

(25000, 20000)

In [7]:
train_data_features #행렬 데이터

<25000x20000 sparse matrix of type '<class 'numpy.int64'>'
	with 2762315 stored elements in Compressed Sparse Row format>

In [15]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
print(vocab[:10])

with open('data/p1_vocab.p', 'wb') as file:
    pickle.dump(vocab, file)

20000
['aag', 'aaron', 'ab', 'abandon', 'abbey', 'abbi', 'abbot', 'abbott', 'abc', 'abduct']


In [9]:
# 벡터화 된 피처를 확인해 봄
import numpy as np
dist = np.sum(train_data_features, axis=0)
    
for tag, count in zip(vocab, dist):
    print(count, tag)
    
pd.DataFrame(dist, columns=vocab)

[[26 48 22 ... 59 40 23]] aag


,aag,aaron,ab,abandon,abbey,abbi,abbot,abbott,abc,abduct,...,zombi bloodbath,zombi film,zombi flick,zombi movi,zone,zoo,zoom,zorro,zu,zucker
0,26,48,22,288,24,30,29,30,125,55,...,23,52,37,89,161,31,71,59,40,23


In [11]:
pd.DataFrame(train_data_features[:10].toarray(), columns=vocab).head()

,aag,aaron,ab,abandon,abbey,abbi,abbot,abbott,abc,abduct,...,zombi bloodbath,zombi film,zombi flick,zombi movi,zone,zoo,zoom,zorro,zu,zucker
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Test 데이터 벡터화

In [12]:
# 테스트 데이터를 벡터화 함
%time test_data_features = pipeline.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

CPU times: user 6.69 s, sys: 58 ms, total: 6.75 s
Wall time: 6.76 s


In [13]:
import pickle

with open('data/p1_train_vector.p', 'wb') as file:
    pickle.dump(train_data_features, file)
    
with open('data/p1_test_vector.p', 'wb') as file:
    pickle.dump(test_data_features, file)
    
